In [1]:
import numpy as np
import matplotlib.pyplot as plt
import optics
import tifffile
from tqdm import tqdm
import utils

def plot_fourier(fourier):
    plt.imshow(np.log(1+np.abs(fourier)))

In [2]:
import napari
viewer = napari.Viewer()

<h1> 1. Field retrieval from hologram </h1>

In [309]:
background = tifffile.imread('background.tif')
sample = tifffile.imread('sample.tif')

N = background[0].shape[0]
Z = background.shape[0]

In [295]:
viewer.add_image(sample)

<Image layer 'sample [1]' at 0x15dbb36f6b0>

In [311]:
m = 1
cm = 1e-2
mm = 1e-3
um = 1e-6
nm = 1e-9

# Laser configuration
lam = 532 * nm

# Camera configuration
dx_cam = 3.5 * um
B_cam = 1 / 2 / dx_cam
dv_cam = 2 * B_cam / N

# Objective lens configuration
NA = 1.2
dx_ol = lam / 4 / NA
B_ol = 1 / 2 / dx_ol
dv_ol = 2 * B_ol / N

# dx_ol = 0.083 * um
# B_ol = 1 / 2 / dx_ol
# dv_ol = 2 * B_ol / N

cutoff = 1/3

<h3> object_center gives frequency coordinate of illumination beam in fourier space </h3>

\begin{gather*}


I(\vec{r}) = |R+U|^2 \\
\hat{I(\vec{\nu})} = \mathcal{F} [I(\vec{r})]\\
\mathcal{P} [\hat{I(\vec{\nu})}] = center[cut[\hat{I(\vec{\nu})}]]= center [\mathcal{F} [R^* U]] = \mathcal{F} [U] \\
\Downarrow \\
U(\vec{r}) = \mathcal{F}^{-1} [\mathcal{F} [U]] 


\end{gather*}

In [329]:
def get_optical_parameters(image_shape, lam, n_medium, dx_cam, dx_ol):
    N = image_shape[1]
    parameters = dict()
    
    # optical parameters
    parameters['lam'] = lam
    
    # Camera parameters
    parameters['dx_cam'] = dx_cam
    parameters['B_cam'] = 1 / 2 / dx_cam
    parameters['dv_cam'] = 2 * parameters['B_cam'] / N
    
    # Objective lens parameters
    parameters['dx_ol'] = dx_ol
    parameters['B_ol'] = 1 / 2 / dx_ol
    parameters['dv_ol'] = 2 * B_ol / N
    
    # Light parameters
    parameters['v0'] = 1 / lam
    parameters['v_nm'] = parameters['v0'] * n_medium
    parameters['k_nm'] = 2 * np.pi * parameters['v_nm']
    parameters['s_nm'] = parameters['v_nm'] // parameters['dv_ol']
    
    return parameters

In [330]:
parameters = get_optical_parameters(image_shape=(N,N), lam=532*nm, NA=1.2, n_medium=1, dx_cam=3.5*um, dx_ol=110*nm)
parameters

NameError: name 'B_cam' is not defined

<h3> Get object field U : F[(DC term) + UR* + U*R] -> F[U] </h3>

In [312]:
temp_background_object_field, temp_sample_object_field = optics.Holography_Off_Axis.get_object_field(background_hologram=background, 
                                                                                           sample_hologram=sample)

100%|██████████| 50/50 [00:06<00:00,  7.72it/s]


In [313]:
background_object_field = temp_background_object_field / temp_background_object_field
sample_object_field = temp_sample_object_field / temp_background_object_field

In [315]:
v0 = 1 / lam
n_medium = 1
v_nm = v0 * n_medium
k_nm = 2 * np.pi * v_nm
s_nm = v_nm // dv_ol
s_nm

213.0

In [316]:
import utils

s0x = []
s0y = []
s0z = []

for i in range(Z):
    sample_freq = np.array(utils.get_maxindex(np.abs(np.fft.fft2(temp_sample_object_field[i]))))
    if i==0:
        center = sample_freq
    incident_freq = sample_freq-center
    s0x.append(incident_freq[1])
    s0y.append(incident_freq[0])
    s0z.append(round(np.sqrt(s_nm**2 - incident_freq[1]**2 - incident_freq[0]**2)))

In [317]:
s_x = np.arange(-N//2, N//2)
s_y = np.arange(-N//2, N//2)
S_x, S_y = np.meshgrid(s_x, s_y)

S_z = s_nm**2 - S_x**2 - S_y**2
S_z[S_z<0] = 0
S_z = np.sqrt(S_z)


<h1> The first Born </h3>

In [318]:
born_scattered_field = sample_object_field - 1

In [320]:
H = 300
born = np.zeros((N, N, H), dtype=np.complex128)
count = np.zeros((N, N, H))
NA_circle = utils.circular_filter((N, N), pixel_radius=int(s_nm))
for i in tqdm(range(Z)):
    # Shifted coordinates
    shifted_S_x = np.roll(S_x, shift=-s0x[i], axis=1)
    shifted_S_y = np.roll(S_y, shift=-s0y[i], axis=0)
    shifted_S_z = np.roll(S_z, shift=-s0x[i], axis=1)
    shifted_S_z = np.roll(shifted_S_z, shift=-s0y[i], axis=0)
            
    # Scattered field
    Us = born_scattered_field[i].copy()
    fourier_Us = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Us)))
    fourier_Us = fourier_Us * NA_circle
    # fourier_Us = np.roll(fourier_Us, shift=-s0y[i]+1, axis=0)
    # fourier_Us = np.roll(fourier_Us, shift=-s0x[i], axis=1)
            
    # Coefficients
    coeff = 2 * np.pi * dv_ol * shifted_S_z / 2 / np.pi / 1j
            
    # field
    F_born = coeff * fourier_Us
        
    yx_idx = np.where((S_z>0) & (NA_circle != 0) & 
                    (shifted_S_x > -N//2) & (shifted_S_x < N//2) &
                    (shifted_S_y > -N//2) & (shifted_S_y < N//2)
                )
        
    # yx_idx = np.where((S_z>0))
        
    z_idx = np.round(shifted_S_z[yx_idx]-shifted_S_z[N//2, N//2] + H//2).astype(int)
    yxz_idx = (yx_idx[0], yx_idx[1], z_idx)
    count[yxz_idx] += np.ones((N, N, H))[yxz_idx]
            
    born[yxz_idx] += F_born[yx_idx]  
    
born[count==0] = 0
born[count!=0] = born[count!=0]/count[count!=0]

100%|██████████| 50/50 [00:25<00:00,  1.96it/s]


In [321]:
potential = np.fft.ifftshift(np.fft.ifftn(np.fft.fftshift(born)))
viewer.add_image(np.real(potential))

<Image layer 'Image' at 0x15dbfd81b80>

In [222]:
ri = np.sqrt(1 + 4 * np.pi * potential / k_nm**2)
viewer.add_image(np.real(ri))

<Image layer 'Image [1]' at 0x15da469a660>

<h1> The first Rytov </h1>

In [326]:
# Rytov
H = 300
rytov = np.zeros((N, N, H), dtype=np.complex128)
count = np.zeros((N, N, H))
NA_circle = utils.circular_filter((N, N), pixel_radius=int(s_nm))
for i in tqdm(range(Z)):
    
    rytov_field = np.log(np.abs(sample_object_field[i])) + 1j * np.angle(sample_object_field[i])
    
    # Shifted coordinates
    shifted_S_x = np.roll(S_x, shift=-s0x[i], axis=1)
    shifted_S_y = np.roll(S_y, shift=-s0y[i], axis=0)
    shifted_S_z = np.roll(S_z, shift=-s0x[i], axis=1)
    shifted_S_z = np.roll(shifted_S_z, shift=-s0y[i], axis=0)

    # Scattered field
    Us = rytov_field.copy()
    fourier_Us = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Us)))
    fourier_Us = fourier_Us * NA_circle
    # fourier_Us = np.roll(fourier_Us, shift=-s0y[i]+1, axis=0)
    # fourier_Us = np.roll(fourier_Us, shift=-s0x[i], axis=1)
            
    # Coefficients
    coeff = 2 * np.pi * dv_ol * shifted_S_z / 2 / np.pi / 1j
            
    # field
    F_rytov = coeff * fourier_Us
        
    yx_idx = np.where((S_z>0) & (NA_circle != 0) & 
                (shifted_S_x > -N//2) & (shifted_S_x < N//2) &
                (shifted_S_y > -N//2) & (shifted_S_y < N//2)
                )
        
    # yx_idx = np.where((S_z>0))
        
    z_idx = np.round(shifted_S_z[yx_idx]-shifted_S_z[N//2, N//2] + H//2).astype(int)
    yxz_idx = (yx_idx[0], yx_idx[1], z_idx)
    count[yxz_idx] += np.ones((N, N, H))[yxz_idx]
            
    rytov[yxz_idx] += F_rytov[yx_idx]
    
rytov[count==0] = 0
rytov[count!=0] = rytov[count!=0]/count[count!=0]

100%|██████████| 50/50 [00:25<00:00,  1.94it/s]


In [327]:
potential = np.fft.ifftshift(np.fft.ifftn(np.fft.fftshift(rytov)))
viewer.add_image(np.real(potential))

<Image layer 'Image [3]' at 0x15db81b5370>

In [307]:
ri = np.sqrt(1 + 4 * np.pi * np.real(potential) / k_nm**2)
viewer.add_image(np.real(ri))

<Image layer 'ri' at 0x15dc5486c00>

<h1> Matlab code - direct immigration </h1>

In [ ]:
phase = np.zeros((Z, N, N))
amplitude = np.zeros((Z, N, N))

for i in tqdm(range(Z)):
    phase[i] = restoration.unwrap_phase(np.angle(retrieved_field[i]))
    amplitude[i] = np.abs(retrieved_field[i])

100%|██████████| 50/50 [00:14<00:00,  3.54it/s]


In [ ]:
illumination_y_idx = np.zeros(Z)
illumination_x_idx = np.zeros(Z)
y_0 = 0
x_0 = 0

for i in range(Z):
    F_back = np.fft.fft2(retrieved_background[i])
    # find illumination center
    yx = get_maxindex(F_back)
    # normal illumination
    if i==0:
        y_0 = yx[0]
        x_0 = yx[1]
    
    # obliqueness
    illumination_y_idx[i] = y_0-yx[1]
    illumination_x_idx[i] = x_0-yx[0]

In [ ]:
n_medium = 1.33
v0_nm = (1/lam) * n_medium

v0_x = dv * np.array(illumination_x_idx)
v0_y = dv * np.array(illumination_y_idx)
v0_z = np.real(np.sqrt(v0_nm**2 - v0_x**2 - v0_y**2))

In [ ]:
fourier_x = np.arange(-N//2, N//2) * dv
fourier_y = np.arange(-N//2, N//2) * dv
fourier_z = np.arange(-height//2, height//2) * dv

Fourier_X, Fourier_Y = np.meshgrid(fourier_x, fourier_y)
Fourier_Z = np.sqrt(Fourier_X**2 + Fourier_Y**2)

fourier_y = fourier_y[:, None]

v3 = v0_nm**2 - Fourier_Z**2
v3[v3<0] = 0
v3 = np.sqrt(v3)
plt.imshow(v3)

In [ ]:
mask = Fourier_Z.copy()
mask[mask > 2*NA/lam] = 0
mask = mask.astype(bool)

rytov = np.zeros((N, N, height), dtype=np.complex128)
count = np.zeros((N, N, height))

In [ ]:
filter = circular_filter((N, N), pixel_radius=int(v_optical/dv/2))

for i in tqdm(range(Z)):
    F_rytov = np.log(amplitude[i]) + 1j*phase[i]
    Us_rytov = np.fft.fftshift(np.fft.fft2(F_rytov)) * dx * dx
    Us_rytov = np.roll(Us_rytov, int(v0_y[i]/dv), axis=0)
    Us_rytov = np.roll(Us_rytov, int(v0_x[i]/dv), axis=1)
    Us_rytov = Us_rytov * filter
        
    size_check = np.zeros((N, N))
    vz = v3 + size_check
    vx = fourier_x + size_check
    vy = fourier_y + size_check
        
    Kx = vx - v0_x[i]
    Ky = vy - v0_y[i]
    Kz = vz - v0_z[i]
        
    Uprime = (vz/1j) * Us_rytov

    xind = np.where(
            ((vz > 0) * filter.astype(bool)) &
            (Kx > (dv * (-np.floor(N / 2)))) &
            (Ky > (dv * (-np.floor(N / 2)))) &
            (Kz > (dv_z * (-np.floor(height / 2)))) &
            (Kx < (dv * (np.floor(N / 2) - 1))) &
            (Ky < (dv * (np.floor(N / 2) - 1))) &
            (Kz < (dv_z * (np.floor(height / 2) - 1)))
        )
    xind = np.ravel_multi_index((xind[0], xind[1]), Uprime.shape)

    # 이 밑에가 ewald sphere 만드는부분
    Uprime = Uprime.flat[xind]
    Kx = Kx.flat[xind]
    Ky = Ky.flat[xind]
    Kz = Kz.flat[xind]

    Kx = np.round(Kx / dv + np.floor(N / 2) + 1).astype(int)
    Ky = np.round(Ky / dv + np.floor(N / 2) + 1).astype(int)
    Kz = np.round(Kz / dv_z + np.floor(height / 2) + 1).astype(int)

    Kzp = np.ravel_multi_index((Ky, Kx, Kz), count.shape)
        
    temp = rytov.flat[Kzp]
    rytov.flat[Kzp] = temp + Uprime
    count.flat[Kzp] += (Uprime != 0)

# Final computation and normalization
rytov[count > 0] = rytov[count > 0] / count[count > 0] / np.prod(np.array([dv, dv, dv_z]))

100%|██████████| 50/50 [00:06<00:00,  7.76it/s]


In [ ]:
rytov_ifftn = np.fft.ifftn(rytov)
potential = rytov_ifftn * 4 * np.pi
k0 = 1/lam
k = 2 * np.pi * n_medium * k0
ri = n_medium * np.sqrt(1+potential/k**2)

# viewer.add_image(np.real(ri))